In [13]:
import pandas as pd
import numpy as np
import csv
import os

In [14]:
dfGitRev = pd.read_csv("../../fx_git_revision.csv")

In [15]:
dfGitRev['component_id'].loc[:] = None
dfGitRev.head()

/tmp/ipykernel_36674/773015420.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfGitRev['component_id'].loc[:] = None


,commit,path,dir,component_id
0,4e9cf83ee8190497736ac31898e63d477c35a651,accessible/atk/DocAccessibleWrap.h,accessible/src/base/,NaN
1,6a627d58418ee75c46ea1a3ba3556382a7f3ea26,js/src/jsdate.h,js/src/,NaN
2,46d7e7df333f0524ed006fa3372f34f729665df4,accessible/base/AccEvent.h,accessible/src/base/,NaN
3,a79338fb53a2b1bf6ea1f91a1c7b4396d3495ef3,js/src/jsutil.h,js/src/,NaN
4,9d3854b96cfe07bfc6ba257bda16783596fdc99e,js/src/jscntxt.h,js/src/,NaN


In [16]:
dfComp = pd.read_csv("../../fx_component.csv")
dfComp.tail()

,id,name
84,92,enterprisepolicies
85,93,storage
86,94,gpu
87,18,formautofill
88,71,permission


## Label dfGitRev with component_id

In [17]:
import re

# Define a function to update the component_name and component_id columns of a row in df1 based on a match in df2
def update_component(row, dfComp):
    # Create a dictionary mapping component_name to component_id from dfComp
    component_id_mapping = dfComp.set_index('name')['id'].to_dict()
    
    dir_path = row['dir']
    # Skip rows where the dir column is not a string or is empty
    if not isinstance(dir_path, str) or not dir_path:
        return row
    
    dir_path = dir_path.rstrip('/') # Remove if there is any '/' at the end of the path string
    dirs = dir_path.split('/') # Take the directories into an array
    dirs = dirs[::-1] # Reverting the array
    for d in dirs:
        for comp_name, comp_id in component_id_mapping.items():
            if re.match(f"^{comp_name}.*", d):
                row['component_id'] = component_id_mapping[comp_name]
                break
        if row['component_id'] > 0:
            break;
    return row

# Apply the update_component function to each row of df1 and update the component_name and component_id columns
dfGitRev = dfGitRev.apply(lambda r: update_component(r, dfComp), axis=1)

In [18]:
# Print the updated DataFrame
dfGitRevLabeled = dfGitRev.dropna(subset=['component_id'])
dfGitRevLabeled.to_csv('../../fx_git_revision_labeled.csv')

In [19]:
dfGitRevLabeled

,commit,path,dir,component_id
0,4e9cf83ee8190497736ac31898e63d477c35a651,accessible/atk/DocAccessibleWrap.h,accessible/src/base/,61.0
2,46d7e7df333f0524ed006fa3372f34f729665df4,accessible/base/AccEvent.h,accessible/src/base/,61.0
5,61f7867cd224579da37595fdac5caeaee9213033,accessible/atk/DocAccessibleWrap.h,accessible/src/base/,61.0
6,6ac35e15fc74320fefc7a6150b59cd2406b1801d,accessible/base/nsAccUtils.h,accessible/src/base/,61.0
7,cc4aafb69233e20cc3b2dde2359c18a44dab825c,accessible/base/Role.h,accessible/src/base/,61.0
...,...,...,...,...
5537611,d3e048885d36557a685b7e304fffd0661ea91e33,accessible/atk/DocAccessibleWrap.cpp,accessible/src/base/,61.0
5537612,d8d0bcab6524fbe176e72d28040be6e2ece785a7,accessible/atk/DocAccessibleWrap.cpp,accessible/src/base/,61.0
5537613,265e6721798a455604328ed5262f430cfcc37c2f,accessible/atk/DocAccessibleWrap.h,accessible/src/base/,61.0
5537616,2bf7aa7227beef25d52463798ffab1cedc4df42c,accessible/atk/DocAccessibleWrap.h,accessible/src/base/,61.0


In [20]:
dfGitRevLabeled['component_id'].isnull().sum()

0

In [21]:
#import re
#
## Define a function to extract the third last word of a string
#def extract_third_last_word(s):
#    if isinstance(s, str):
#        words = re.findall(r'\b\w+\b', s)
#        if len(words) > 2:
#            return words[-3]
#    return ''
#
## Define a function to update the component_name and component_id columns of a row in df1 based on a match in df2
#def update_component(row):
#    # Skip rows where the dir column is not a string or is empty
#    if not isinstance(row['dir'], str) or not row['dir']:
#        return row
#    # Look for a match in df2 based on the third last word of the directory path in df1
#    match = df2[df2['name'].str.contains(extract_third_last_word(row['dir']))]
#    if not match.empty and pd.isnull(row['component_id']):
#        # If a match is found and the component_id is NaN, update the component_name and component_id columns
#        row['component_name'] = match.iloc[0]['name']
#        row['component_id'] = match.iloc[0]['id']
#    return row
#
## Apply the update_component function to each row of df1 and update the component_name and component_id columns
#df1 = df1.apply(update_component, axis=1)
#
## Print the updated DataFrame
#print(df1)